In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from time import time

In [ ]:
# tensor =tf.constant([[1,2,3],[4,5,6]])

# tensor.numpy()

In [ ]:
df_pagamentos= pd.read_csv('/home/alessandro/tcc/dados/payment_data.csv')
df_pagamentos.set_index('id', inplace=True) # seta o id como index
df_consumidores = pd.read_csv('/home/alessandro/tcc/dados/customer_data.csv')
df_consumidores.set_index('id', inplace=True) # seta o id como index

In [ ]:
dados_consumidores=df_consumidores.join(df_pagamentos, how='inner') # junta os dois dataframes
dados_consumidores.columns

In [ ]:
colunas=['alvo',
 'recurso_1',
 'recurso_2',
 'recurso_3',
 'recurso_4',
 'recurso_5',
 'recurso_6',
 'recurso_7',
 'recurso_8',
 'recurso_9',
 'recurso_10',
 'recurso_11',
 'atraso_t1',
 'atraso_t2',
 'atraso_t3',
 'atraso_total',
 'pagemento_normal',
 'codigo_produto',
 'limite_credito',
 'data_atualizada',
 'saldo_atual_produto',
 'saldo_mais_alto_produto',
 'report_date']


dados_consumidores.columns=colunas
dados_consumidores.to_excel("")

In [ ]:
dados_consumidores['alvo'].value_counts() # conta quantas vezes cada label aparece

In [ ]:
dados_consumidores.isnull().sum() # conta quantos valores faltantes

In [ ]:
dados_consumidores['codigo_produto'].value_counts() # conta quantas vezes cada categoria aparece

In [ ]:
for prod_code in dados_consumidores["codigo_produto"].unique():
    
    dados_consumidores["codigo_produto{}".format(prod_code)] = dados_consumidores["codigo_produto"] == prod_code
    
dados_consumidores



In [ ]:
dados_consumidores['recurso_4'].unique()

In [ ]:
for feature_id in [1, 3, 5, 6, 7, 9]:
    for value in dados_consumidores["recurso_{}".format(feature_id)].unique():
        dados_consumidores["recurso_{}_{}".format(feature_id, value)] = dados_consumidores["recurso_{}".format(feature_id)] == value
dados_consumidores


In [ ]:
dados_consumidores = dados_consumidores.drop(["limite_credito", "report_date", "data_atualizada", "codigo_produto", "recurso_1", "recurso_3", "recurso_5", "recurso_6", "recurso_7", "recurso_9"], axis=1)

In [ ]:
dados_consumidores_array_np = dados_consumidores.to_numpy(na_value=np.nan).astype(float)

In [ ]:


#imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

imputer = SimpleImputer(verbose=2)
data = imputer.fit_transform(dados_consumidores_array_np)

# (dados_consumidores==np.nan).any()
# print(imp_mean)

In [ ]:
scaler = MinMaxScaler()

dados_consumidores_normalizado =scaler.fit_transform(data)

dados_teste =scaler.fit_transform(data)


In [ ]:
# balanceado o base de dados
mask = dados_consumidores_normalizado[:, 0] == 1
risk_customers = dados_consumidores_normalizado[mask]
print("numero de clientes com risco de crédito alto = {}".format(risk_customers.shape[0]))

safe_customers = dados_consumidores_normalizado[~mask]
indices = np.random.choice(safe_customers.shape[0], risk_customers.shape[0]*2, replace=False)
safe_customers = safe_customers[indices]
print("numero de clientes com risco de crédito baixo = {}".format(safe_customers.shape[0]))

dados_consumidores_normalizado = np.concatenate([risk_customers, safe_customers], axis=0)
print("tamanho do novo dataframe = ", dados_consumidores_normalizado.shape)

In [ ]:

X=dados_consumidores_normalizado[:,1:]  # features
y=dados_consumidores_normalizado[:,0] #label 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=10)



In [ ]:

def create_model(optimizer='adam'):

    model =tf.keras.models.Sequential()

    model.add(tf.keras.layers.Dense(units=64, input_shape=(69,)))
    model.add(tf.keras.layers.Dense(36, activation='relu'))
    model.add(tf.keras.layers.Dense(18, activation='relu'))
    model.add(tf.keras.layers.Dense(9, activation='relu'))

    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


    model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])

    return model

model=create_model()
# model = KerasClassifier(build_fn=create_model, verbose=1)

# optimizers = ['rmsprop','adam']
# epochs = np.array([10])
# batches = np.array([32])

# param_grid= dict(optimizer=optimizers, epochs=epochs, batch_size=batches)
# grid= GridSearchCV(estimator=model, param_grid=param_grid)

# start=time()
# grid_result = grid.fit(X_train, y_train)
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# for params, mean_score, scores in grid_result.grid_scores_:
#     print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))
# print("total time:",time()-start)

In [ ]:
model.fit(X_train, y_train, epochs=500, batch_size=32, verbose=0)
model.evaluate(X_test, y_test, verbose=0)

In [ ]:


yk=model.predict(X_test)
score=model.evaluate(X_test, y_test)
resposta=np.array([1 if yk[i]>0.5 else 0 for i in range(len(yk))])
bons_pagadores=resposta==0 

print("Bom pagador",resposta[bons_pagadores].shape[0])
print("Mau pagador",resposta[~bons_pagadores].shape[0])


resp=np.array(resposta).reshape(-1,1)
alvo=np.array(y_test).reshape(-1,1)
comparativo=np.append(alvo,resp,axis=1)


df_saida=pd.DataFrame(comparativo, columns=['alvo','previsto'])

# df=pd.DataFrame(data=resposta, index=y_test, columns=['label'])
# df.loc[df['label']==1,'label']='Mau pagador'
# df.loc[df['label']==0,'label']='Bom pagador'
df_saida.to_excel('resultado.xlsx', index=False)

In [ ]:
# # balance the dataset
# mask = dados_teste[:, 0] == 1

# risk_customers = dados_teste[mask]
# print("number of risk customers = {}".format(risk_customers.shape[0]))

# safe_customers = dados_teste[~mask]
# indices = np.random.choice(safe_customers.shape[0], risk_customers.shape[0]*2, replace=False)

# safe_customers = safe_customers[indices]
# print("number of safe customers = {}".format(safe_customers.shape[0]))

# data = np.concatenate([risk_customers, safe_customers], axis=0)
# print("Shape of selected data = ", dados_teste.shape)ll